In [1]:
# 用于获取最新的DTBS的A区中的 cz 纯债价值 pd 平底溢价率
# 由于原先的cz是每周获取一次，因此若需要每日更新一次则需要这个文件，考虑到wind数据容量有限选择增量更新
# cz这个数据感觉更新的相当晚，因此不要太早跑，安全起见5点之后甚至第二天跑

import pandas as pd
import numpy as np
import pickle
import time
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
with open(DTBS_path, 'rb') as f:
    DTBS = pickle.load(f)

with open('database_ByZCG\\DCBS_DataFrame.pkl', 'rb') as f:  
    DCBS_DataFrame = pickle.load(f)
    

with open('database_ByZCG\\DTBS_DataFrame.pkl', 'rb') as f:  
    DTBS_DataFrame = pickle.load(f)

# 只提取A区中存活的转债的时间序列信息
A_DF = DTBS_DataFrame['A']
A_DF = A_DF[A_DF['ia']==1].copy()

In [2]:
# 找出cz为空的那些行
df = A_DF[A_DF['cz'].isnull()]

day_list = sorted(list(set(df['date'])))

need_query_mapping = {}

for day in day_list:
    need_query_mapping[day] = list(df[df['date']==day]['bond_code'])

need_query_mapping

{'2023-09-06': ['110088.SH',
  '113545.SH',
  '113631.SH',
  '123128.SZ',
  '123156.SZ',
  '127027.SZ',
  '127061.SZ',
  '110047.SH',
  '110063.SH',
  '110070.SH',
  '110087.SH',
  '110089.SH',
  '110092.SH',
  '111001.SH',
  '111002.SH',
  '111003.SH',
  '111007.SH',
  '111012.SH',
  '113025.SH',
  '113027.SH',
  '113046.SH',
  '113047.SH',
  '113058.SH',
  '113059.SH',
  '113064.SH',
  '113534.SH',
  '113535.SH',
  '113566.SH',
  '113591.SH',
  '113600.SH',
  '113615.SH',
  '113621.SH',
  '113623.SH',
  '113632.SH',
  '113636.SH',
  '113638.SH',
  '113640.SH',
  '113641.SH',
  '113649.SH',
  '113650.SH',
  '113653.SH',
  '113657.SH',
  '113663.SH',
  '118000.SH',
  '118005.SH',
  '118006.SH',
  '118008.SH',
  '118010.SH',
  '118018.SH',
  '118020.SH',
  '118022.SH',
  '118023.SH',
  '118028.SH',
  '118029.SH',
  '123012.SZ',
  '123013.SZ',
  '123031.SZ',
  '123046.SZ',
  '123052.SZ',
  '123076.SZ',
  '123078.SZ',
  '123085.SZ',
  '123091.SZ',
  '123106.SZ',
  '123118.SZ',
  '123124.S

In [3]:
from WindPy import w
w.start()

for day in day_list:
    
    bond_code_list = need_query_mapping[day]
    
    # 纯债价值列表
    bond_value_list = w.wsd(bond_code_list, "strbvalue", day, day, "").Data[0]
    
    print(bond_value_list)
    
    for bond_code_index in range(len(bond_code_list)):
        bond_code = bond_code_list[bond_code_index]
        bond_value = bond_value_list[bond_code_index]
        DTBS['A'][bond_code][day]['cz'] = bond_value
        DTBS['A'][bond_code][day]['pd'] = (DTBS['A'][bond_code][day]['csv'] / DTBS['A'][bond_code][day]['cz'] - 1) * 100

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
[95.937, 106.6433, 100.4994, 99.4457, 82.1489, 103.8797, 94.2587, 111.4849, 107.5406, 106.9297, 96.4427, 97.9752, 94.7969, 94.4104, 100.6823, 92.9853, 89.2961, 80.0112, 107.2141, 95.0954, 102.6565, 101.7768, 99.3009, 98.7452, 97.0052, 103.3111, 106.518, 101.7946, 103.0167, 99.6679, 104.5874, 102.3571, 103.2527, 101.0163, 94.1637, 100.4772, 92.9548, 97.576, 90.7679, 90.6129, 90.6725, 85.6192, 88.5525, 98.9604, 89.1315, 85.446, 90.6984, 78.8066, 77.4041, 86.6614, 95.768, 89.1094, 88.3894, 75.4736, 101.6532, 103.4714, 97.7656, 101.3192, 101.7242, 98.5962, 103.1587, 93.8337, 103.6904, 102.1666, 90.025, 81.7377, 95.4795, 85.2745, 91.8997, 84.809, 95.9123, 89.0636, 81.1699, 88.9744, 85.7245, 107.7642, 116.596, 103.7078, 103.5796, 98.982, 100.4161, 100.6635, 103.3

[95.7732, 106.4681, 100.2812, 99.2778, 82.012, 103.5969, 94.0872, 111.3376, 107.2955, 106.5656, 96.1749, 97.6996, 94.6133, 94.2728, 100.5022, 92.822, 89.121, 79.8721, 106.9632, 94.9895, 102.3988, 101.534, 99.1069, 98.5484, 96.8148, 103.191, 106.4902, 101.6148, 102.8118, 99.4663, 104.3943, 102.1869, 103.0989, 100.8435, 94.0161, 100.2946, 92.7955, 97.3131, 90.5979, 90.4426, 90.501, 85.4526, 88.3738, 98.7942, 88.981, 85.2994, 90.5286, 78.6764, 77.3168, 86.5659, 95.576, 88.9344, 88.2575, 75.3437, 101.6037, 103.4399, 97.6254, 101.1648, 101.5274, 98.3888, 102.9438, 93.6521, 103.499, 101.99, 89.8431, 81.5738, 95.2978, 85.1209, 91.7338, 84.6704, 95.7214, 88.8871, 81.0341, 88.7958, 85.5514, 107.4554, 116.2524, 103.4256, 103.2727, 98.6834, 100.1018, 100.4104, 103.3199, 104.0302, 101.661, 95.968, 95.9544, 99.7375, 96.6954, 91.8759, 91.6815, 96.4699, 84.7591, 97.5778, 97.4139, 89.6486, 80.243, 100.6297, 105.4377, 114.161, 105.5838, 97.1242, 106.0498, 63.1895, 105.1292, 103.907, 105.1295, 104.0495,

In [4]:
DTBS['A']['113545.SH']['2023-08-29']

{'cpr': 40.8,
 'dp': 118.749,
 'dl': 159.54899999999998,
 'ytm': -2.23,
 'bl': 9.865,
 'trt': 2.49,
 'yl': 2.129,
 'csp': 9.96,
 'ia': 1,
 'xx': 0,
 'hs': 0,
 'csv': 84.33734939759036,
 'qs': 0,
 'qs15': 0,
 'qs30': 0,
 'xx15': 3,
 'xx30': 3,
 'cz': 106.7029,
 'pd': -20.960583641503316}

In [5]:
# 保存上面的这些信息的更改
f_save = open(DTBS_path, 'wb')
pickle.dump(DTBS, f_save)
f_save.close()